In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=4f768a6a78a5bae90e4502ba161f5e744c884a0755e6dc87cf4378e26da299a1
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [ ]:
import pandas as pd
import re
import unicodedata
import swifter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

space_remove = re.compile(r'\s+')

def load_process_data(filepath):
  df = pd.read_csv(filepath)

  df['comment'] = df['comment'].swifter.apply(lambda x: unicodedata.normalize('NFKC', x))
  df['comment'] = df['comment'].swifter.apply(lambda x: space_remove.sub(' ', x))

  label_encoder = LabelEncoder()
  df['label'] = label_encoder.fit_transform(df['label'])

  x_train, x_test, y_train, y_test = train_test_split(df['comment'], df['label'], stratify=df['label'], test_size=0.2, random_state=42)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train, test_size=0.25, random_state=42)

  return{
      'x_train': x_train, 'x_val': x_val, 'x_test': x_test, 'y_train': y_train, 'y_val': y_val, 'y_test': y_test,
      'label_encoder': label_encoder, 'value_counts': df['label'].value_counts()
      }


data = load_process_data('/content/drive/MyDrive/dataset.csv')

Pandas Apply:   0%|          | 0/44001 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/44001 [00:00<?, ?it/s]

In [ ]:
print(data)

{'x_train': 42032    লিজেন্ড ম্যান❤❤❤আপনার সবগুলো নাটকই দেখি।।।অসাধ...
42552    পরি আপু তোমার ছবিতে এত খারাপ কমেন্ট কেন আসে তু...
41479    এখনই উচিত নিজের স্বার্থে পরিবারের স্বার্থে দেশ...
34279                                              তর হেদা
15947    নুস্রাতের ধর্ষন কারী সিরাউদ্দোলা আর আর এই নাস্...
                               ...                        
999      পরকালে বিশ্বাস করো না, আবার আল্লাহ বিশ্বাস করো...
11316                         মগা কবির,,,,ছাগল,,পাগল,গাধা,
3876     গরু জবাই আর কয় বার হয় বলেন?? কথা এমন একটা জি...
25852                যে ইসলামের সান্তি বুঝবেনা সেই নাস্তিক
25119    মাগীর পুরি, তর বাপেরে তুই দেখছিলি তুই যখন কনডম...
Name: comment, Length: 26400, dtype: object, 'x_val': 25898    মাগী তোর নাটক আর দেখমো না নিজের জনপ্রিয় তাটা ...
37518    জায়েদ খান অরে কে চিনে ৷শালা ত্রকটা অহংকারি লো...
32419    বইন তোমাকেও জাতি ছাগল মুক্তাদিরের মতো বাঁশ মের...
43759                   রহস্যময় পৃথিবীর অদ্ভুত বালক tui O
29686                                       শয়তা

In [ ]:
import torch
from torch.utils.data import Dataset

class BanglaDataset(Dataset):
    def __init__(self, comment, label, tokenizer, max_len=128):
        self.comment   = comment
        self.label     = label
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.comment)

    def __getitem__(self, index):
        comment = str(self.comment.iloc[index])
        label   = int(self.label.iloc[index])

        enc = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }


In [ ]:
from torch.utils.data import WeightedRandomSampler, DataLoader


def data_loaders(x, y, tokenizer, class_counts, batch_size, sampler=True):
  dataset = BanglaDataset(x, y, tokenizer)
  if sampler and 'train' in x.name:
    class_weights = 1. / torch.tensor(class_counts.values, dtype=torch.float)
    sample_weights = class_weights[y]
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    return DataLoader(dataset, batch_size=batch_size, sampler=sampler)
  else:
    return DataLoader(dataset, batch_size=batch_size)


In [ ]:

from transformers import AutoConfig, BertForSequenceClassification

def initialize_model(num_classes, dropout=0.1):
    config = AutoConfig.from_pretrained(
        "sagorsarker/bangla-bert-base" ,
        num_labels = num_classes,
        hidden_dropout_prob = dropout,
        attention_probs_dropout_prob = dropout
    )

    return BertForSequenceClassification.from_pretrained(
        "sagorsarker/bangla-bert-base" ,
        config=config
    )


In [ ]:
print(len(data['label_encoder'].classes_))

5


In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

def create_optimizer_and_scheduler(model, train_loader, learning_rate, weight_decay, warmup_steps, num_epochs):

    optimizer = AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    return optimizer, scheduler

In [ ]:
from tqdm import tqdm

def train_epoch(model, train_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [ ]:
import numpy as np

def evaluate_model(model, dataloader, device):
    model.eval()
    preds, labels, probs = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            logits = outputs.logits.detach().cpu().numpy()
            preds.extend(np.argmax(logits, axis=1))
            labels.extend(batch['labels'].cpu().numpy())
            probs.extend(torch.softmax(outputs.logits, dim=1).cpu().numpy())

    return preds, labels, probs

In [ ]:
!pip install Optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 28.4 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import f1_score, precision_recall_curve
import optuna
from optuna.pruners import MedianPruner

def objective(trial, data, tokenizer):

  params = {
      'learning_rate': trial.suggest_float('learning_rate', 1e-6, 5e-5, log=True),
      'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32]),
      'epochs': trial.suggest_int('epochs', 3, 10),
      'dropout': trial.suggest_float('dropout', 0.1, 0.3),
      'weight_decay': trial.suggest_float('weight_decay', 1e-6, 0.1, log=True),
      'warmup_steps': trial.suggest_int('warmup_steps', 0, 500)
  }

  model = initialize_model(num_classes=len(data['label_encoder'].classes_), dropout=params['dropout'])

  train_loader = data_loaders(data['x_train'], data['y_train'], tokenizer, data['value_counts'], params['batch_size'])
  val_loader = data_loaders(data['x_val'], data['y_val'], tokenizer, data['value_counts'], params['batch_size'], sampler=False)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  optimizer, scheduler = create_optimizer_and_scheduler(
    model, train_loader, params['learning_rate'], params['weight_decay'], params['warmup_steps'], params['epochs']
  )

  for epoch in range(params['epochs']):
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)

    val_preds, val_labels, _ = evaluate_model(model, val_loader, device)
    f1 = f1_score(val_labels, val_preds, average='weighted')

    trial.report(f1, epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return f1





In [ ]:
def run_hyperparameter_tuning(data, tokenizer, n_trials=10):

    study = optuna.create_study(
        direction='maximize',
        pruner=MedianPruner(n_warmup_steps=2)
    )
    study.optimize(
        lambda trial: objective(trial, data, tokenizer),
        n_trials=n_trials,
        timeout=3600
    )
    return study.best_params

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
best_params = run_hyperparameter_tuning(data, tokenizer, n_trials=10)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

[I 2025-05-19 08:58:29,456] A new study created in memory with name: no-name-7ace8517-52f0-4474-8e91-4027fbbca551


model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training: 100%|██████████| 3300/3300 [11:53<00:00,  4.62it/s]
[I 2025-05-19 09:11:39,310] Trial 0 finished with value: 0.826955829362729 and parameters: {'learning_rate': 1.675780943616927e-05, 'batch_size': 8, 'epochs': 6, 'dropout': 0.1467021947863662, 'weight_decay': 1.8209485537751343e-05, 'warmup_steps': 310}. Best is trial 0 with value: 0.826955829362729.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training: 100%|██████████| 3

In [ ]:
print(best_params)

{'learning_rate': 1.675780943616927e-05, 'batch_size': 8, 'epochs': 6, 'dropout': 0.1467021947863662, 'weight_decay': 1.8209485537751343e-05, 'warmup_steps': 310}


In [ ]:
def optimize_thresholds(model, dataloader, num_classes, device):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            probs = torch.softmax(outputs.logits, dim=1)
            all_probs.append(probs.cpu().numpy())
            all_labels.append(batch['labels'].cpu().numpy())

    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)

    thresholds = []
    for class_idx in range(num_classes):
        precision, recall, thresh = precision_recall_curve(
            (all_labels == class_idx).astype(int),
            all_probs[:, class_idx]
        )
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        best_thresh = thresh[np.argmax(f1_scores)]
        thresholds.append(best_thresh)

    return thresholds

In [ ]:
from sklearn.metrics import classification_report

def train_final_model(data, tokenizer, best_params):

    model = initialize_model(len(data['label_encoder'].classes_), best_params['dropout'])
    train_loader = data_loaders(
        data['x_train'], data['y_train'], tokenizer,
        data['value_counts'], best_params['batch_size']
    )
    val_loader = data_loaders(
        data['x_val'], data['y_val'], tokenizer,
        data['value_counts'], best_params['batch_size'], sampler=False
    )
    test_loader = data_loaders(
        data['x_test'], data['y_test'], tokenizer,
        data['value_counts'], best_params['batch_size'], sampler=False
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    optimizer, scheduler = create_optimizer_and_scheduler(
        model, train_loader, best_params['learning_rate'],
        best_params['weight_decay'], best_params['warmup_steps'], best_params['epochs']
    )

    best_f1 = 0
    for epoch in range(best_params['epochs']):
        train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)

        val_preds, val_labels, _ = evaluate_model(model, val_loader, device)
        f1 = f1_score(val_labels, val_preds, average='weighted')
        print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Val F1: {f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), 'best_model.pt')

    model.load_state_dict(torch.load('best_model.pt'))

    thresholds = optimize_thresholds(model, val_loader, len(data['label_encoder'].classes_), device)
    print("Optimal thresholds:", thresholds)

    test_preds, test_labels, _ = evaluate_model(model, test_loader, device)
    print("\nTest Classification Report:")
    print(classification_report(test_labels, test_preds, target_names=data['label_encoder'].classes_))

    return model, thresholds

In [ ]:
model, thresholds = train_final_model(data, tokenizer, best_params)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training: 100%|██████████| 3300/3300 [11:51<00:00,  4.64it/s]


Epoch 1 - Loss: 0.7040, Val F1: 0.8196


Training: 100%|██████████| 3300/3300 [11:50<00:00,  4.64it/s]


Epoch 2 - Loss: 0.4312, Val F1: 0.8326


Training: 100%|██████████| 3300/3300 [11:50<00:00,  4.64it/s]


...

Epoch 4 - Loss: 0.2011, Val F1: 0.8324


Training: 100%|██████████| 3300/3300 [11:50<00:00,  4.65it/s]


Epoch 5 - Loss: 0.1279, Val F1: 0.8354


Training: 100%|██████████| 3300/3300 [11:50<00:00,  4.65it/s]


Epoch 6 - Loss: 0.0866, Val F1: 0.8340
Optimal thresholds: [np.float32(0.51315683), np.float32(0.607372), np.float32(0.59258014), np.float32(0.4878922), np.float32(0.4240994)]

Test Classification Report:
              precision    recall  f1-score   support

   not bully       0.85      0.86      0.85      3068
   religious       0.92      0.91      0.92      1515
      sexual       0.84      0.84      0.84      1786
      threat       0.81      0.77      0.79       339
       troll       0.77      0.77      0.77      2093

    accuracy                           0.84      8801
   macro avg       0.84      0.83      0.83      8801
weighted avg       0.84      0.84      0.84      8801



In [ ]:
import json, joblib, os

out_dir = "models/bert-bn-v1"
os.makedirs(out_dir, exist_ok=True)

model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)

def to_py(x):
    if isinstance(x, np.generic):
        return x.item()
    if isinstance(x, np.ndarray):
        return x.tolist()
    return x

thresholds_py = [to_py(v) for v in thresholds]

with open(os.path.join(out_dir, "thresholds.json"), "w") as f:
    json.dump(thresholds_py, f)


Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to models/bert-bn-v1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabu

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json

model_dir = "models/bert-bn-v1"

model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

with open(f"{model_dir}/thresholds.json") as f:
    thresholds = json.load(f)

In [1]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 798.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.4 MB/s eta 0:00:00


In [23]:
import torch
from pathlib import Path

def export_to_onnx(model, tokenizer, output_path):
    model.eval()

    dummy_input = tokenizer(
        "এই একটি উদাহরণ বাক্য",
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)

    torch.onnx.export(
        model,
        (dummy_input["input_ids"], dummy_input["attention_mask"]),
        output_path,
        input_names=["input_ids", "attention_mask"],
        output_names=["logits"],
        dynamic_axes={
            'input_ids': {0: 'batch_size', 1: 'sequence_length'},
            'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
            'logits': {0: 'batch_size'}
        },
        opset_version=17,
        do_constant_folding=True
    )

    return output_path

In [24]:
from transformers import BertTokenizer, AutoModelForSequenceClassification

model_path = '/content/drive/MyDrive/bert-bn-v1'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

onnx_file = export_to_onnx(model, tokenizer, "bangla_bert.onnx")
print("Exported:", onnx_file)


Exported: bangla_bert.onnx


In [26]:
!ls /content/drive/MyDrive/ | grep 'bangla_bert'


bangla_bert.onnx
